In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [0]:
watermark_ts = spark.sql("select * from otc.silver.watermark where table_name='order_items' ").select('watermark_ts').collect()[0][0]

eligible_order_items_records = spark.sql(f"""
                                      select * from otc.bronze.src_order_items
                                      where ingest_ts > TIMESTAMP '{watermark_ts}'
                                      """)

# Dedup in silver
eligible_order_items_records = eligible_order_items_records.withColumn('SequenceOfRecord', F.row_number().over( Window.partitionBy('order_id', 'order_item_id').orderBy( F.desc('ingest_ts') ) ) )\
    .filter( F.col('SequenceOfRecord') == 1)\
        .drop('SequenceOfRecord')

eligible_order_items_records.display()
eligible_order_items_records.createOrReplaceTempView("eligible_order_items_records")

In [0]:
%sql
SELECT order_item_id, order_id, product_id, quantity, unit_price, updated_at, ingest_ts FROM eligible_order_items_records

In [0]:
%sql

INSERT INTO otc.silver.order_items (order_item_id, order_id, product_id, quantity, unit_price, updated_at, ingest_ts)
SELECT order_item_id, order_id, product_id, quantity, unit_price, updated_at, ingest_ts FROM eligible_order_items_records

In [0]:
%sql
UPDATE otc.silver.watermark
  SET watermark_ts = (select max(ingest_ts) from otc.silver.order_items)
  WHERE table_name = 'order_items'

In [0]:
%sql
select * from otc.silver.order_items

In [0]:
%sql
select * from otc.silver.watermark